<a href="https://colab.research.google.com/github/Caramel96/Samuel_BuckbyCE888/blob/main/lab_4%20/%20my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np 
import sklearn

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', header=None
)

In [4]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [19]:
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,7.33,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,2.82,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,4.42,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,8.50,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-

In [5]:
df = df.drop([0], axis=1)

In [6]:
arr = df

In [7]:
rated = arr.replace(99, np.nan)

In [8]:
def replace(orig, percentage=0.1):
  new_data = orig.copy()
  rated = np.where(~np.isnan(orig))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [9]:
new_arr, idx = replace(rated.values, 0.1)

In [10]:
rated.values[idx[0][0], idx[1][0]]

3.25

In [11]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [12]:
n_latent_factors = 2
user_ratings = new_arr
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1],n_latent_factors))

In [13]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=10):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating) and rating!=99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
            mse_history.append(mse)
    return mse_history

In [14]:
num_iter = 10
hist = sgd(num_iter)  # Note how the MSE decreases with the number of iterations
plt.figure()
plt.plot(np.arange(0, num_iter, 10), hist)
plt.xlabel("Iterations")
plt.ylabel("MSE")
plt.show()

Iteration 0/10:	MSE=26.248091


KeyboardInterrupt: ignored

In [15]:
predictions= latent_user_preferences.dot(latent_item_features.T)
predictions

array([[0.23883033, 0.14924643, 0.0858186 , ..., 0.13104367, 0.16722272,
        0.24932904],
       [0.94014394, 0.50285501, 0.36578986, ..., 0.5976558 , 0.68670235,
        1.13604685],
       [0.72885144, 0.42459987, 0.2720953 , ..., 0.42974198, 0.52069173,
        0.81725178],
       ...,
       [0.48024718, 0.2159707 , 0.20036796, ..., 0.34482458, 0.36452413,
        0.6550063 ],
       [0.86369168, 0.57163907, 0.29980434, ..., 0.44305497, 0.59401245,
        0.84338073],
       [1.00648768, 0.58901457, 0.37485881, ..., 0.59085537, 0.71813649,
        1.12367709]])

In [16]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

In [ ]:
comparison_data

In [17]:
#Changing the number of latent factors while all the other hyper-parameters stay the same
n_latent_factors2=4
user_ratings2= new_arr
latent_user_preferences2=np.random.random((user_ratings2.shape[0], n_latent_factors2))
latent_item_features2=np.random.random((user_ratings2.shape[1],n_latent_factors2))

predictions2 = latent_user_preferences2.dot(latent_item_features2.T)
values2 = [zip(user_ratings2[i], predictions2[i]) for i in range(predictions2.shape[0])]
comparison_data2 = pd.DataFrame(values2)
comparison_data2.columns = df.columns

In [18]:
comparison_data2

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, 0.7542141188974626)","(99.0, 0.5093824550436737)","(-9.66, 1.0860089973817344)","(-8.16, 0.9524198926112662)","(-7.52, 0.6007565682939876)","(-8.5, 0.7047233569785274)","(-9.85, 1.0881851314696256)","(4.17, 0.6522044542094076)","(-8.98, 0.486933627965916)","(-4.76, 0.29747685725760226)","(-8.5, 0.7390153932143133)","(-6.75, 0.9804599593137718)","(-7.18, 0.8807497976154798)","(8.45, 0.9020098086998094)","(-7.18, 0.5037295981664289)","(-7.52, 0.4162023800249622)","(-7.43, 0.9181993418941443)","(-9.81, 0.8545342931760189)","(-9.85, 0.14383407808814783)","(-9.85, 0.9313784571066495)","(-9.37, 0.9399633590345073)","(1.5, 0.94693069439134)","(-4.37, 0.45721267340822797)","(-9.81, 0.6332648265646734)","(-8.5, 0.9109479028075983)","(1.12, 0.6197751346223199)","(7.82, 1.0665969026684048)","(2.86, 0.8067316122576463)","(9.13, 0.6972432341607926)","(-7.43, 1.1582617345338244)","(2.14, 0.6203004201922049)","(-4.08, 1.1584540105648216)","(-9.08, 0.6361538814542638)","(7.82, 1.102211059648368)","(5.05, 0.4862175894484489)","(4.95, 1.0118421836093694)","(-9.17, 0.5653191506713248)","(-8.4, 0.7188155986063913)","(-8.4, 0.8319098879372969)","(-8.4, 0.6964057480077499)",...,"(8.59, 0.6150065528542769)","(99.0, 0.5634428513511146)","(-6.84, 0.7108943732052222)","(-9.03, 0.4847250487984674)","(99.0, 0.5538718121076769)","(-1.36, 0.5902956902073029)","(-9.08, 0.5293978270449213)","(8.3, 0.7710494815553267)","(5.68, 0.7623999151048243)","(-4.81, 0.8850106782264001)","(nan, 0.7774752548273343)","(nan, 0.39939567555750216)","(nan, 0.7094009062952256)","(nan, 0.6616527797638916)","(nan, 0.6900358823650752)","(nan, 0.40318679913043826)","(nan, 0.8350863467723577)","(-9.42, 0.7772431684442256)","(nan, 0.8364209275708414)","(nan, 0.4276563910449352)","(nan, 0.49747562647065485)","(-7.72, 1.1245554678634635)","(nan, 0.8780879370934579)","(nan, 0.45416142479123417)","(nan, 0.17134651258881975)","(nan, 0.6516714029744936)","(nan, 0.28312793640161743)","(nan, 0.4292688855423963)","(nan, 0.728083484904006)","(nan, 1.0494367595933976)","(2.82, 0.40725824777629493)","(nan, 0.8902185161311403)","(nan, 0.5084484357847205)","(nan, 0.9251104223477361)","(nan, 0.5675622346155672)","(nan, 0.8659095842274823)","(-5.63, 0.3407948542051149)","(nan, 0.7025234701741822)","(nan, 0.38376761999608705)","(nan, 0.4713238000837292)"
1,"(4.08, 0.7677022440752559)","(-0.29, 0.5902729226871704)","(6.36, 1.3797963512312046)","(4.37, 1.2315851848227986)","(-2.38, 1.1207040640167332)","(-9.66, 1.3064767197357716)","(-0.73, 1.2053464769274216)","(-5.34, 1.2440192020020704)","(8.88, 0.7750590242874746)","(9.22, 0.6506985729755737)","(6.75, 0.9924747038450126)","(8.64, 1.4127188291544603)","(99.0, 1.0230811325023965)","(7.43, 1.3685486157981233)","(4.56, 0.46541232822983664)","(-0.97, 0.8861373180877719)","(4.66, 1.467378873196121)","(-0.68, 1.2135144051309197)","(3.3, 0.3850951465160101)","(-1.21, 1.0390110006690396)","(0.87, 1.140286960849603)","(8.64, 1.1259930064571277)","(8.35, 1.0011526216107005)","(9.17, 1.1603105377697993)","(0.05, 1.49784117885025)","(99.0, 1.1451361809569602)","(4.71, 1.311363294094057)","(99.0, 0.7970352632226899)","(-0.39, 1.3441947604979754)","(6.99, 1.5473744920540458)","(6.5, 1.0275853200536138)","(-0.92, 1.6247553411617943)","(99.0, 0.8415524663262136)","(9.03, 1.1190768205492034)","(-1.8, 0.809000508930465)","(0.73, 1.4330116647470794)","(7.09, 0.8305819944995093)","(3.4, 0.8791058214263203)","(-0.87, 1.033809448284763)","(7.91, 1.2568052139009591)",...,"(-6.7, 0.9746422354381676)","(-3.35, 0.6759707609304229)","(-9.03, 0.801497527234969)","(4.47, 1.269150048968943)","(4.08, 0.8343386304290912)","(-3.83, 0.9559636113583669)","(8.74, 0.893493737647426)","(1.12, 0.8728619206603133)","(0.78, 1.2249893773002691)",

In [ ]:
# What's the average rating of the worst-rated joke in the Jokes dataset? Give your answer with two decimal values (e.g., 1.23).
rated.mean(axis=0).min()

-3.8338796373689923

In [ ]:
#in the movies dataset, what's the rating that user 12622 has given to joke 16?
#Give your answer with two decimal values (e.g., 0.13).
rated.iloc[12622,16]

8.25